In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv("C:\\Users\\Sanjana\\Downloads\\Crop Yiled with Soil and Weather.csv")

In [3]:
df=pd.DataFrame(data)

# EDA

In [4]:
df.head()

,Fertilizer,temp,N,P,K,yeild
0,80.0,28.0,80.0,24.0,20.0,12.0
1,77.0,27.0,78.0,23.0,20.0,12.0
2,80.0,26.0,80.0,24.0,20.0,12.0
3,80.0,28.0,80.0,24.0,20.0,12.0
4,78.0,27.0,78.0,23.0,19.0,12.0


In [5]:
df.tail()

,Fertilizer,temp,N,P,K,yeild
2591,78.942684,27.014634,76.838312,23.087606,20.044206,10.348503
2592,79.077362,27.153127,76.927657,23.068234,20.155436,10.528122
2593,78.986271,28.108948,77.164352,22.863654,20.989871,10.564205
2594,78.894767,28.141344,76.925149,23.071553,20.873662,10.442511
2595,79.050307,26.980865,77.124758,23.050197,19.960695,10.584638


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2596 entries, 0 to 2595
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Fertilizer  2596 non-null   float64
 1   temp        2596 non-null   float64
 2   N           2596 non-null   float64
 3   P           2596 non-null   float64
 4   K           2596 non-null   float64
 5   yeild       2596 non-null   float64
dtypes: float64(6)
memory usage: 121.8 KB


In [7]:
df.describe()

,Fertilizer,temp,N,P,K,yeild
count,2596.000000,2596.000000,2596.000000,2596.000000,2596.000000,2596.000000
mean,66.487433,33.848237,69.522900,20.708194,17.806268,8.533832
std,9.747669,5.371279,6.802806,1.973419,1.940037,1.938789
min,49.751436,23.771310,58.839466,17.723223,14.704883,5.150745
25%,59.865423,28.000000,64.834635,18.988224,15.975932,6.929404
50%,65.011969,36.965121,69.894076,20.889579,18.061975,8.501720
75%,76.943112,38.928914,76.845919,22.125618,19.133858,10.045179
max,80.223893,40.272480,80.218705,25.162178,22.064666,12.337651


In [8]:
df.isnull().sum()

Fertilizer    0
temp          0
N             0
P             0
K             0
yeild         0
dtype: int64

In [9]:
Q1_yield = df['yeild'].quantile(0.25)
Q3_yield = df['yeild'].quantile(0.75) 
Q2_yield = df['yeild'].quantile(0.50) 

In [10]:
N_HIGH_THRESHOLD = 69.0     
P_SUFFICIENT = 20.0          
K_SUFFICIENT = 8.0   

In [11]:
N_CRITICAL_LOW = 65.0        
P_CRITICAL_LOW = 19.5        


In [12]:
conditions = [
    
    (df['yeild'] >= Q2_yield) &
    (df['N'] >= N_HIGH_THRESHOLD) &
    (df['P'] >= P_SUFFICIENT) &
    (df['K'] >= K_SUFFICIENT),

    (df['yeild'] <= Q1_yield) |
    (df['N'] < N_CRITICAL_LOW) |
    (df['P'] < P_CRITICAL_LOW)
]

In [13]:
choices = ['Grade A', 'Grade C']

In [14]:
df['Grade'] = np.select(
    condlist=conditions,
    choicelist=choices,
    default='Grade B')

In [15]:
df['Grade'].value_counts()

Grade
Grade C    1155
Grade A    1100
Grade B     341
Name: count, dtype: int64

# feature engineering

In [16]:
from sklearn.preprocessing import MinMaxScaler

In [17]:
cols_to_scale = ['Fertilizer', 'temp', 'N', 'P', 'K']

In [18]:
scaler = MinMaxScaler()

In [19]:
X_scaled = scaler.fit_transform(df[cols_to_scale])

# Encoding

In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
le = LabelEncoder()

In [22]:
y=df['Grade']

In [23]:
y_encoded = le.fit_transform(y)

# train test split

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.20, random_state=42
)

# model training

In [26]:
!pip install xgboost

In [27]:
import xgboost as xgb
model = xgb.XGBClassifier(objective='multi:softmax', 
                          n_estimators=200, 
                          random_state=42, 
                          use_label_encoder=False,
                          eval_metric='mlogloss') 
model.fit(X_train, Y_train)

C:\Users\Sanjana\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [10:24:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [28]:
from sklearn.metrics import classification_report, accuracy_score


In [29]:
Y_pred = model.predict(X_test)

In [30]:
accuracy = accuracy_score(Y_test, Y_pred)

In [31]:
accuracy*100

98.26923076923076

# save

In [32]:
import joblib
joblib.dump(model, 'quality_prediction_model1.joblib')

['quality_prediction_model1.joblib']

In [34]:
import json
model_columns = ['Fertilizer', 'temp', 'N', 'P', 'K']
with open("model_columns1.json", "w") as f:
    json.dump(model_columns, f)